# ARIM-Academy：　機器データ利活用ユースケース
### タイトル：茶の抽出条件に関する機械学習/数学的最適化検討
### 機器：誘導結合プラズマ発光分光分析法（ICP-OES）
### 分析：回帰分析（線形重回帰分析、人工ニューラルネットワーク）


---
## データセット

    
本編で扱う 『茶の元素分析データセット』（data.xlsx）は、ブラックセイロン(BC)、ブラックトルコ(BT)、グリーンセイロン(GC)、グリーントルコ(GT)の4種類の茶葉について、3つの濃度（1%、2%、3%）で抽出した元素分析値の誘導結合プラズマ発光分光分析法（ICP-OES）の元素分析データです。[1]

この研究では、機械学習（回帰分析としてMLR, ANN、および主成分分析、ISOMAP、階層クラスタ分析など）の手法を使い、茶の品種、濃度、浸出時間によるミネラル組成の変化を調査し、特定のミネラルレベルを調整する条件を見つける検討が行われています。

[1]: Durmus, Y., Atasoy, A.D. & Atasoy, A.F. Mathematical optimization of multilinear and artificial neural network regressions for mineral composition of different tea types infusions. Sci Rep 14, 18285 (2024). https://doi.org/10.1038/s41598-024-69149-1

---
<br>  
<img src="./img/main_image.jpg" width="50%">
<br> 

---

**Chemical composition** 
* 9種類の元素分析値（Al, Ca, Cu, Fe,K,Mg,Mn,Na,Zn） 

**Experimental parameters**  
* Concentration：　お茶の濃度
* time:　抽出時間

**Label**
* grouped:　品種・色・濃度・抽出時間を識別できる一種の試料名（MLは使用せず） 
* teaConc:　濃度ラベル（MLは使用せず）
* tea_org:　色ラベル（MLは使用せず）
* tea_var:　品種ラベル（MLは使用せず）
* tea：　　 色と品種ラベル（MLで使用）

---

### 教材への接続
google colabにおけるオンラインの場合にこのラインを実行します。（<font color="red">Google colabに接続しない場合には不要</font>）

In [ ]:
!git clone https://github.com/ARIM-Usecase/Example_2.git
%cd Example_2

---
### ライブラリのインポート
コード実行で必要なpythonのライブラリをimport文でロードします。

In [1]:
# 汎用ライブラリ
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# 機械学習
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

#深層学習
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Input, LeakyReLU
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')

## 1.データセットの読み込みと前処理

### サンプルファイルの読み込み
pandasライブラリのread_excel()関数は、Excelファイルを読み込んでpandasのDataFrame形式に変換する関数です。ここでは[data]フォルダーにあるdata.xlsxファイルをDataFrameとして読み込み、その結果をdfという変数に格納します。168のサンプルに対して16の特徴量からなるデータ行列となっています。

In [2]:
# データの読み込み
df = pd.read_excel('./data/data.xlsx',header = 1)
df

,Al,Ca,Cu,Fe,K,Mg,Mn,Na,Zn,grouped,tea,Concentration,time,teaConc,tea_org,tea_var
0,3.297,4.356,0.031290,0.067,99.06,3.531,1.455,0.541,0.131,Black Turkish 1 2,BT,1,2,BT1,black,turki
1,4.267,4.118,0.031290,0.079,106.50,3.378,1.542,0.603,0.126,Black Turkish 1 2,BT,1,2,BT1,black,turki
2,4.088,4.763,0.033370,0.084,114.00,4.763,1.838,1.058,0.156,Black Turkish 1 5,BT,1,5,BT1,black,turki
3,4.338,4.556,0.033370,0.091,122.60,5.005,2.269,0.958,0.162,Black Turkish 1 5,BT,1,5,BT1,black,turki
4,4.732,5.138,0.035514,0.110,132.40,5.626,2.998,1.510,0.165,Black Turkish 1 10,BT,1,10,BT1,black,turki
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,16.690,8.895,0.153000,0.236,323.40,20.450,10.420,6.360,0.335,Green Ceylan 3 30,GC,3,30,GC3,green,ceylon
164,17.620,8.909,0.177000,0.261,334.20,23.486,11.330,7.133,0.351,Green Ceylan 3 45,GC,3,45,GC3,green,ceylon
165,17.920,9.056,0.180000,0.266,332.30,22.840,11.290,7.609,0.358,Green Ceylan 3 45,GC,3,45,GC3,green,ceylon
166,17.820,9.128,0.175000,0.273,367.30,24.560,12.110,8.537,0.372,Green Ceylan 3 60,GC,3,60,GC3,green,ceylon


### ペアプロット（散布図行列）

茶の元素分析データセットにペアプロットを適用することで、以下の情報を得ることができます。

* **特徴量間の関係**: 異なる特徴量間の関係を視覚的に確認できます。例えば、AlとZnが正の相関関係にあるか、負の相関関係にあるかなどを一目でわかります。
* **クラス間の分離性**: 各品種（ブラックセイロン(BC)、ブラックトルコ(BT)、グリーンセイロン(GC)、グリーントルコ(GT)）のデータがどのように分布しているか、他の品種とどの程度分離しているかを確認できます。
* **外れ値の検出**: データに含まれる外れ値を視覚的に確認することができます。
* **特徴量選択**: どの特徴量が品種の分類に最も貢献しているか、あるいは冗長な特徴量があるかなどを判断する手がかりになります。

In [ ]:
#　ペアプロットによる各変数間の二次元空間像（その２）
sns.pairplot(df,hue='tea')
plt.show()

## 2.　前処理
### カテゴリカル変数のOne Hot Encoding化
ワンホットエンコード（OneHot Encoding）とは、カテゴリカルデータ（質的変数）を数値ベースで表現するための手法です。カテゴリカルデータのようなデータが名前やラベルで示されている場合、数値としての大小関係や加減計算が適用できません。たとえば、「茶の銘柄」が「ブラックセイロン」、「グリーントルコ」、「ブラックトルコ」のようにカテゴリで示される場合、これらをそのまま数値として扱うことは適切ではありません。OneHotエンコードでは、カテゴリごとに専用のバイナリ（0または1）のフラグを持つベクトルに変換します。実際、「茶の銘柄」は以下のようにラベリングされています：

| Index | 銘柄              |
|:--------:|:--------:|
| BC     | ブラックトルコ 　|
| GT     | ブラックセイロン   |
| BT     | グリーントルコ   |
| BT     | グリーンセイロン  |

このデータをOneHotエンコードすると、以下のような形式に変換されます：

| Index | ブラックトルコ |ブラックセイロン  | グリーントルコ |グリーンセイロン | 
|:--------:|:--------:|:--------:|:--------:|:--------:|
| BT     | 1                | 0              | 0              |0    |
| BC     | 0                | 1              | 0              |0    |
| GT     | 0                | 0              | 1              |0    |
| GC     | 0                | 0              | 0              |0    |

このように、各銘柄（カテゴリ）に対応する列が数値として生成され、該当するカテゴリのみ1、それ以外は0で表現されます。これにより、カテゴリカルデータを機械学習モデルに適用できる数値形式に変換できます。 

Scikit-learnではOneHotEncoderクラスを使用して、質的変数のお茶の銘柄「tea」列の値を簡単にワンホットエンコーディング化することができます。

In [3]:
# OneHotエンコーディング
encoder = OneHotEncoder(drop='first', sparse_output=False)
df_encoded = encoder.fit_transform(df[['tea']])

# エンコードされたteaをデータフレームに追加
encoded_columns = encoder.get_feature_names_out(['tea'])
df[encoded_columns] = df_encoded

【解説】**ワンホットエンコード（OneHot Encoding）クラス**  

1. **`OneHotEncoder`のインスタンス作成**  
   `OneHotEncoder`の基本とScikit-learnでの実装方法の解説です。
  
- **drop='first'**: この引数は、1つのカテゴリを基準カテゴリとしてドロップし、他のカテゴリをエンコードします。これにより、マルチコリニアリティ（相関が強すぎる変数）を避け、エンコードされた列の数を減らします。ドロップされなかったカテゴリは相対的に扱われます。

- **sparse_output=False**: 結果をスパース行列（効率的にメモリを使う形式）ではなく、通常のNumPy配列で返すオプションです。

2. **データの変換**  
   `fit_transform`メソッドを使用して、カテゴリカルデータをOneHotエンコードします。

### 説明変数と目的変数
データフレームdfから、茶の濃度（Concentration）と浸出時間（time）に加えて、One Hot Encodingされたお茶の銘柄を説明変数とします。一方、目的変数は元素分析値のミネラル成分一式とします。

In [4]:
# 必要な説明変数と目的変数の抽出
X = df[['Concentration', 'time'] + list(encoded_columns)]
y = df[['Al', 'Ca', 'Cu', 'Fe', 'K', 'Mg', 'Mn', 'Na', 'Zn']]

In [5]:
X

,Concentration,time,tea_BT,tea_GC,tea_GT
0,1,2,1.0,0.0,0.0
1,1,2,1.0,0.0,0.0
2,1,5,1.0,0.0,0.0
3,1,5,1.0,0.0,0.0
4,1,10,1.0,0.0,0.0
...,...,...,...,...,...
163,3,30,0.0,1.0,0.0
164,3,45,0.0,1.0,0.0
165,3,45,0.0,1.0,0.0
166,3,60,0.0,1.0,0.0


### 標準化  
機械学習モデルはデータのスケールに敏感であるため、データを標準化することが重要です。標準化とは、各特徴量を平均0、標準偏差1にスケーリングする手法で、異なるスケールを持つ特徴量を均等に扱えるようにします。これにより、モデルが特徴量のスケールに影響されず、データ全体の構造を正確に反映することが可能になります。

標準化は、pythonではScikit-learnライブラリで提供されている`StandardScaler`クラスで、データの標準化を簡単に行うことができます。

In [6]:
# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### データセットの分割（Data set splitting）
機械学習モデルの性能を正確に評価するためには、モデルを学習させる**訓練データ**と、学習済みのモデルの性能を評価するための**テストデータ**にデータを分割する必要があります。scikit-learnのtrain_test_split()関数を使用すると、この分割を簡単に実行できます。ここでの引数は`test_size=0.2`として訓練データ：0.8、テストデータ：0.2とします。

In [7]:
# 教師データと検証データの分割
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                                                    test_size=0.2, 
                                                    random_state=42
                                                    )

## 3.機械学習（予測モデル）
### 3.1 scikit-learnによる予測モデルの選択（MLR）
LinearRegression()クラスは、Scikit-learnライブラリが提供するクラスで、予測型による線形重回帰モデルを構築するために使用されます。

### モデル構築と評価
forループを活用して、9つの目的変数(y)に格納されているミネラル成分ごとに多重線形回帰（Multiple Linear Regression, MLR）モデルを構築し、その性能を評価します。モデル評価には、決定係数（R²）、RMSE（平方根平均二乗誤差）、およびMAE（平均絶対誤差）の3つの指標を用いています。目的変数に対するモデルの性能指標（R2、RMSE、MAE）、および学習された回帰係数（Coef）と切片（Intercept）を`mlr_results`変数に格納します。

In [8]:
# 多重線形回帰（MLR）のモデル構築と評価
mlr_model = LinearRegression()

mlr_results = {}

for mineral in y.columns:
    mlr_model.fit(X_train, y_train[mineral])

    y_pred = mlr_model.predict(X_test)
    
    r2 = r2_score(y_test[mineral], y_pred)
    rmse = np.sqrt(mean_squared_error(y_test[mineral], y_pred))
    mae = mean_absolute_error(y_test[mineral], y_pred)
    
    mlr_results[mineral] = {'R2': r2, 'RMSE': rmse, 'MAE': mae, 'Coef': mlr_model.coef_, 'Intercept': mlr_model.intercept_}

### 評価結果の表示
mlr_results変数に格納されている各ミネラル成分ごとの多重線形回帰（MLR）モデルの結果を各目的変数（ミネラル成分）ごとに表示します。

In [9]:
# MLRの結果表示
for mineral, result in mlr_results.items():
    print(f'{mineral} - Coefficients: {result["Coef"]}, Intercept: {result["Intercept"]}')
    print(f'R2: {result["R2"]}, RMSE: {result["RMSE"]}, MAE: {result["MAE"]}\n')

Al - Coefficients: [3.79023937 1.43132072 2.422908   1.4915705  3.00820455], Intercept: 8.332939758068418
R2: 0.7860598485225262, RMSE: 2.095182608221989, MAE: 1.6405828680427006

Ca - Coefficients: [0.82831951 0.50689865 0.60567955 0.94928797 0.35196316], Intercept: 6.063938541567816
R2: 0.9316957963319135, RMSE: 0.31652730807384644, MAE: 0.24580753656596274

Cu - Coefficients: [0.0158984  0.00961836 0.00235734 0.02024182 0.00080977], Intercept: 0.062010195124226244
R2: 0.6022344943444129, RMSE: 0.023018847988192325, MAE: 0.017072223559960718

Fe - Coefficients: [0.031093   0.03127944 0.04095397 0.04086792 0.03604626], Intercept: 0.1409555974041486
R2: 0.7745619365593756, RMSE: 0.024588217217625982, MAE: 0.02036783627774086

K - Coefficients: [ 84.6439924   38.63003728 -10.61593612 -47.01264766 -30.12526335], Intercept: 242.08273098181968
R2: 0.9066364327056077, RMSE: 28.73240614126902, MAE: 23.273625727622917

Mg - Coefficients: [ 5.38707208  4.35945176 -1.60994223 -3.04017305 -1.916

### 3.2 kerasによる予測モデルの選択（ANN）
次に人工ニューラルネットワーク（Artificial Neural Network, ANN）モデルを用いて複数のミネラル成分を予測する方法について解説します。ANNのアーキテクチャは以下の通りで、この構成をKerasのSequential APIを使って実装します。ここでは、各ミネラル成分を個別に予測するため、それぞれの成分に対してANNモデルを個別に構築します。

<br>  
<img src="./img/image2.png" width="50%">
<br> 

Kerasを用いて人工ニューラルネットワーク（ANN）モデルを構築するbuild_ann_moedel()関数を定義し、都度、呼び出すようにします。

In [14]:
# モデル構築の関数化
def build_ann_model(input_dim):
    model = Sequential()
    model.add(Input(shape=(input_dim,))) 
    model.add(LeakyReLU())
    model.add(Dense(18))
    model.add(LeakyReLU())
    model.add(Dense(8))
    model.add(LeakyReLU())
    model.add(Dense(12))
    model.add(LeakyReLU())
    model.add(Dense(18))
    model.add(LeakyReLU())
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(learning_rate=0.0112), loss='mse')
    
    return model

入力次元（input_dim）を指定すると、18ノードの入力層、8ノードと12ノードの中間層（隠れ層）、および18ノードの出力層が順次追加されます。各層には非線形活性化関数としてLeakyReLUが適用され、最後に単一出力の回帰モデルが形成されます。モデルはAdamオプティマイザとmse（平均二乗誤差）損失関数でコンパイルされています。この関数により、柔軟に構築されたANNモデルを返します。

In [15]:
ann_model = build_ann_model(X_train.shape[1])
ann_model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ leaky_re_lu_40 (LeakyReLU)      │ (None, 5)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_40 (Dense)                │ (None, 18)             │           108 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_41 (LeakyReLU)      │ (None, 18)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_41 (Dense)                │ (None, 8)              │           152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_42 (LeakyReLU)      │ (None, 8)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_42 (Dense)                │ (None, 12)             │           108 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_43 (LeakyReLU)      │ (None, 12)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 18)             │           234 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_44 (LeakyReLU)      │ (None, 18)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_44 (Dense)                │ (None, 1)              │            19 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 621 (2.43 KB)

 Trainable params: 621 (2.43 KB)

 Non-trainable params: 0 (0.00 B)

### 解説：

このコードは、人工ニューラルネットワーク（ANN）モデルを使って複数のミネラルを予測するために、各ミネラルに対してANNモデルを個別に構築し、訓練しています。以下に、コードの各部分を解説します。

`Sequential`モデルを関数にまとめることで、コードの再利用性を向上させることができます。以下は、Kerasの`Sequential`モデルを構築する部分を関数化したコードです。
- `build_ann_model(input_dim)` 関数：
  - 入力次元を引数として受け取り、指定された構造のANNモデルを構築します。
  - `Dense`レイヤーと`LeakyReLU`アクティベーション関数を順に追加し、最後に出力層を作成しています。
  - モデルは`Adam`オプティマイザーとMSE損失関数を使ってコンパイルされます。


#### ANNモデルの構築
```python
  ann_model = Sequential()
  ann_model.add(Input(shape=(input_dim,))) 
  ann_model.add(LeakyReLU())
```
- `Sequential()`：モデルを順に層を積み重ねるためのKerasの基本的なモデル構造です。
- `Input(shape=(input_dim,))`：全結合層（Dense層）を追加しています。入力次元は特徴量の数に一致します（input_dim = `X_train.shape[1]` で入力層の次元数を指定）。
- `LeakyReLU()`：活性化関数としてLeaky ReLUを使用しています。ReLUは負の入力に対して0を出力する一方、Leaky ReLUはわずかな負のスロープを持ち、学習が進みやすくなる利点があります。

#### 隠れ層の追加
```python
    model.add(Dense(18))
    model.add(LeakyReLU())
    model.add(Dense(8))
    model.add(LeakyReLU())
    model.add(Dense(12))
    model.add(LeakyReLU())
    model.add(Dense(18))
    model.add(LeakyReLU())
```
- モデルには4つの隠れ層があり、それぞれ18, 8、12、18個のニューロンを持ちます。各層の後には `LeakyReLU` が活性化関数として追加されています。これにより、ネットワークが非線形性を学習しやすくなっています。

#### 出力層
```python
ann_model.add(Dense(1))
```
- 最終層は1つのニューロンを持つ出力層です。ここで、単一の予測値（各ミネラルの濃度）を出力します。

#### モデルのコンパイル
```python
ann_model.compile(optimizer=Adam(learning_rate=0.0112), loss='mse')
```
- `Adam(learning_rate=0.0112)`：Adamオプティマイザーを使用し、学習率を0.0112に設定しています。Adamは学習効率が高く、一般的にANNでよく使用される最適化手法です。
- `loss='mse'`：損失関数として平均二乗誤差（MSE）を使用しています。これは回帰問題に適した損失関数です。
  

### モデル構築と評価
各ミネラルについて、事前に定義したbuild_ann_model関数でモデルを構築し、トレーニングデータを用いて100エポック訓練します。テストデータに対する予測値から、決定係数（R²）、平方根平均二乗誤差（RMSE）、および平均絶対誤差（MAE）を算出し、結果を辞書ann_resultsに保存します。このプロセスは各ミネラル成分に対して繰り返されます。

In [16]:
# ANNモデルの構築と評価
ann_results = {}

# 各ミネラルに対してモデルを訓練
for mineral in y.columns:
    ann_model = build_ann_model(X_train.shape[1])
    ann_model.fit(X_train, y_train[mineral], epochs=100, verbose=0)

    y_pred = ann_model.predict(X_test)

    r2 = r2_score(y_test[mineral], y_pred)
    rmse = np.sqrt(mean_squared_error(y_test[mineral], y_pred))
    mae = mean_absolute_error(y_test[mineral], y_pred)

    ann_results[mineral] = {'R2': r2, 'RMSE': rmse, 'MAE': mae}

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


【解説】
- モデルを訓練データ `X_train` と各ミネラルのターゲットデータ `y_train[mineral]` を使用して学習させます。
- `epochs=100`：100エポック分、モデルを繰り返し訓練します。
- `verbose=0`：訓練中の出力を抑制しています。

このようにして、各ミネラルに対してANNモデルが構築・訓練され、それぞれの予測結果が`ann_results`に保存されることが想定されています。

### 評価結果の表示
ann_results変数に格納されている各ミネラル成分ごとの人工ニューラルネットワーク（ANN）モデルの結果を各目的変数（ミネラル成分）ごとに表示させます。

In [17]:
# ANNの結果表示
for mineral, result in ann_results.items():
    print(f'{mineral} - R2: {result["R2"]}, RMSE: {result["RMSE"]}, MAE: {result["MAE"]}\n')

Al - R2: 0.9533437299651805, RMSE: 0.9784322046959174, MAE: 0.6958892866022448

Ca - R2: 0.9209241140015427, RMSE: 0.34057244044381757, MAE: 0.2723775706796085

Cu - R2: 0.7926877388558237, RMSE: 0.016618144081716628, MAE: 0.011838683974816518

Fe - R2: 0.9019128353173091, RMSE: 0.016218829527059716, MAE: 0.013470030325669381

K - R2: 0.8025480199066556, RMSE: 41.78438599466305, MAE: 33.40428255866556

Mg - R2: 0.9879400774985075, RMSE: 0.8017636954392116, MAE: 0.6413118534873514

Mn - R2: 0.9443344211892712, RMSE: 0.6507738481566887, MAE: 0.4146259260738596

Na - R2: 0.9879566819706869, RMSE: 0.16684922834778276, MAE: 0.13362499506333292

Zn - R2: 0.9323192928798181, RMSE: 0.020328060920087137, MAE: 0.016349280714988708



### モデル性能の考察

多重線形回帰（MLR）モデルと人工ニューラルネットワーク（ANN）モデルを比較すると、ANNモデルは全ての指標においてMLRモデルを上回る性能を示しています。まず、決定係数（R²）についてみてみると、ANNモデルのR²は、すべてのミネラル成分で0.89以上を記録しており、特にAl（0.98）やCa（0.97）などで非常に高い値を示しています。一方、MLRモデルではR²が0.60〜0.93の範囲で、CuやZnのような要因では予測性能が低下しています。この結果は、ANNが非線形な関係性を捉える能力を持つため、MLRでは捉えきれないデータの複雑なパターンを学習できることを示しています。
